In [1]:
%run Runge_Adams_FDA.ipynb

In [2]:
mu, t, h = symbols(r'\mu, t, h', real=True)
u, u1 = symbols('u, u1', cls=Function)

In [3]:
eq = u(t).diff(t, 2) + mu*(u(t)**2 - 1)*u(t).diff(t) + u(t)
eq

\mu*(u(t)**2 - 1)*Derivative(u(t), t) + u(t) + Derivative(u(t), (t, 2))

In [4]:
init((u(t), u1(t)), t, h)
set_clip(8, 7, Rational(0, 1))

In [5]:
expand((T(u(t+h) - u(t-h)))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [6]:
def CompactAdamsBashforth5(g, y):
#     return expand(-10*g(y)/3 - 22*g(y.subs(t, t+h))/3 + 8*g(y.subs(t, t+2*h))/3)
    return expand(-10*g(y)/3 - 22*g(y.subs(t, t+h))/3 + 8*g(y.subs(t, t+2*h))/3  + (-9*y + 9*y.subs(t, t+h))/h)

def CompactAdamsBashforth7(g, y):
    return expand(-87*g(y)/11 - 651*g(y.subs(t, t+h))/11 - 387*g(y.subs(t, t+2*h))/11\
        + 45*g(y.subs(t, t+3*h))/11) + (-28*y - 475*y.subs(t, t+h)/11 + 783*y.subs(t, t+2*h)/11)/h

In [7]:
def g(y):
    return Matrix([\
        y[1],\
        -mu*(y[0]**2 - 1)*y[1] - y[0],\
    ])

def f(y):
    return Matrix([\
        clip_n(y[1]),\
        clip_n(-mu*(y[0]**2 - 1)*y[1] - y[0]),\
    ])
    
s = (('u(t)', 'y[0]'), ('u1(t)', 'y[1]'), (r'\mu', 'mu'))
def replace(r):
    for a, b in s:
        r = r.replace(a, b)
    return r

In [8]:
start = time.time()

## Gauss–Legendre method

In [9]:
set_clip(5, 4, Rational(0, 1))
r = GaussLegendre2(f, Matrix([u(t), u1(t)]))

In [10]:
r

Matrix([
[                                u1(t)],
[-\mu*u(t)**2*u1(t) + \mu*u1(t) - u(t)]])

In [11]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


u_{tt}/2

h^2 =>


u_{ttt}/6

h^3 =>


u_{tttt}/24

- u_{1} + u_{t}
+h\left(
\frac{u_{tt}}{2}
\right)
+h^2\left(
\frac{u_{ttt}}{6}
\right)
+h^3\left(
\frac{u_{tttt}}{24}
\right)


In [12]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


u1_{tt}/2

h^2 =>


u1_{ttt}/6

h^3 =>


u1_{tttt}/24

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{u1_{tt}}{2}
\right)
+h^2\left(
\frac{u1_{ttt}}{6}
\right)
+h^3\left(
\frac{u1_{tttt}}{24}
\right)


In [13]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-\mu*u1*(u - 1)*(u + 1)/2 - u/2

h^2 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/3 - \mu*u*(u**2 - 2*u1**2 - 1)/3 + u1/3

h^3 =>


-\mu**3*u1*(u - 1)**3*(u + 1)**3/4 - \mu**2*u*(u - 1)*(u + 1)*(3*u**2 - 16*u1**2 - 3)/12 + \mu*u1*(8*u**2 - u1**2 - 3)/6 + u/4

- u_{1} + u_{t}
+h\left(
- \frac{\mu u_{1} \left(u - 1\right) \left(u + 1\right)}{2} - \frac{u}{2}
\right)
+h^2\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{3} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{3} + \frac{u_{1}}{3}
\right)
+h^3\left(
- \frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{4} - \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(3 u^{2} - 16 u_{1}^{2} - 3\right)}{12} + \frac{\mu u_{1} \left(8 u^{2} - u_{1}^{2} - 3\right)}{6} + \frac{u}{4}
\right)


In [14]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


\mu**2*u1*(u - 1)**2*(u + 1)**2/2 + \mu*u*(u**2 - 2*u1**2 - 1)/2 - u1/2

h^2 =>


\mu**3*u1*(u - 1)**3*(u + 1)**3/3 + \mu**2*u*(u - 1)*(u + 1)*(u**2 - 5*u1**2 - 1)/3 - \mu*u1*(10*u**2 - u1**2 - 4)/6 - u/3

h^3 =>


\mu**4*u1*(u - 1)**4*(u + 1)**4/4 + \mu**3*u*(u - 1)**2*(u + 1)**2*(3*u**2 - 26*u1**2 - 3)/12 - \mu**2*u1*(31*u**4 - 20*u**2*u1**2 - 40*u**2 + 4*u1**2 + 9)/12 - \mu*u*(4*u**2 - 8*u1**2 - 3)/6 + u1/4

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{2} + \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{2} - \frac{u_{1}}{2}
\right)
+h^2\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{3} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 5 u_{1}^{2} - 1\right)}{3} - \frac{\mu u_{1} \left(10 u^{2} - u_{1}^{2} - 4\right)}{6} - \frac{u}{3}
\right)
+h^3\left(
\frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{4} + \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(3 u^{2} - 26 u_{1}^{2} - 3\right)}{12} - \frac{\mu^{2} u_{1} \left(31 u^{4} - 20 u^{2} u_{1}^{2} - 40 u^{2} + 4 u_{1}^{2} + 9\right)}{12} - \frac{\mu u \left(4 u^{2} - 8 u_{1}^{2} - 3\right)}{6} + \frac{u_{1}}{4}
\right)


In [15]:
replace(str(f2[2].collect(mu)))

'mu**3*(y[0]**6*y[1]/3 - y[0]**4*y[1] + y[0]**2*y[1] - y[1]/3) + mu**2*(y[0]**5/3 - 5*y[0]**3*y[1]**2/3 - 2*y[0]**3/3 + 5*y[0]*y[1]**2/3 + y[0]/3) + mu*(-5*y[0]**2*y[1]/3 + y[1]**3/6 + 2*y[1]/3) - y[0]/3'

## Crank–Nicolson method

In [16]:
set_clip(5, 4, Rational(0, 1))
r = CrankNicolson2(f, Matrix([u(t), u1(t)]))

In [17]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


u_{tt}/2

h^2 =>


u_{ttt}/6

h^3 =>


u_{tttt}/24

- u_{1} + u_{t}
+h\left(
\frac{u_{tt}}{2}
\right)
+h^2\left(
\frac{u_{ttt}}{6}
\right)
+h^3\left(
\frac{u_{tttt}}{24}
\right)


In [18]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


u1_{tt}/2

h^2 =>


u1_{ttt}/6

h^3 =>


u1_{tttt}/24

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{u1_{tt}}{2}
\right)
+h^2\left(
\frac{u1_{ttt}}{6}
\right)
+h^3\left(
\frac{u1_{tttt}}{24}
\right)


In [19]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-\mu*u1*(u - 1)*(u + 1)/2 - u/2

h^2 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/3 - \mu*u*(u**2 - 2*u1**2 - 1)/3 + u1/3

h^3 =>


-\mu**3*u1*(u - 1)**3*(u + 1)**3/4 - \mu**2*u*(u - 1)*(u + 1)*(3*u**2 - 16*u1**2 - 3)/12 + \mu*u1*(8*u**2 - u1**2 - 3)/6 + u/4

- u_{1} + u_{t}
+h\left(
- \frac{\mu u_{1} \left(u - 1\right) \left(u + 1\right)}{2} - \frac{u}{2}
\right)
+h^2\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{3} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{3} + \frac{u_{1}}{3}
\right)
+h^3\left(
- \frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{4} - \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(3 u^{2} - 16 u_{1}^{2} - 3\right)}{12} + \frac{\mu u_{1} \left(8 u^{2} - u_{1}^{2} - 3\right)}{6} + \frac{u}{4}
\right)


In [20]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


\mu**2*u1*(u - 1)**2*(u + 1)**2/2 + \mu*u*(u**2 - 2*u1**2 - 1)/2 - u1/2

h^2 =>


\mu**3*u1*(u - 1)**3*(u + 1)**3/3 + \mu**2*u*(u - 1)*(u + 1)*(u**2 - 5*u1**2 - 1)/3 - \mu*u1*(10*u**2 - u1**2 - 4)/6 - u/3

h^3 =>


\mu**4*u1*(u - 1)**4*(u + 1)**4/4 + \mu**3*u*(u - 1)**2*(u + 1)**2*(3*u**2 - 26*u1**2 - 3)/12 - \mu**2*u1*(31*u**4 - 20*u**2*u1**2 - 40*u**2 + 4*u1**2 + 9)/12 - \mu*u*(4*u**2 - 8*u1**2 - 3)/6 + u1/4

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{2} + \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{2} - \frac{u_{1}}{2}
\right)
+h^2\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{3} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 5 u_{1}^{2} - 1\right)}{3} - \frac{\mu u_{1} \left(10 u^{2} - u_{1}^{2} - 4\right)}{6} - \frac{u}{3}
\right)
+h^3\left(
\frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{4} + \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(3 u^{2} - 26 u_{1}^{2} - 3\right)}{12} - \frac{\mu^{2} u_{1} \left(31 u^{4} - 20 u^{2} u_{1}^{2} - 40 u^{2} + 4 u_{1}^{2} + 9\right)}{12} - \frac{\mu u \left(4 u^{2} - 8 u_{1}^{2} - 3\right)}{6} + \frac{u_{1}}{4}
\right)


In [21]:
replace(str(f2[2].collect(mu)))

'mu**3*(y[0]**6*y[1]/3 - y[0]**4*y[1] + y[0]**2*y[1] - y[1]/3) + mu**2*(y[0]**5/3 - 5*y[0]**3*y[1]**2/3 - 2*y[0]**3/3 + 5*y[0]*y[1]**2/3 + y[0]/3) + mu*(-5*y[0]**2*y[1]/3 + y[1]**3/6 + 2*y[1]/3) - y[0]/3'

## "original" Runge–Kutta method

In [22]:
set_clip(7, 6, Rational(0, 1))
r = RungeKutta4(f, Matrix([u(t), u1(t)]))

In [23]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


\mu*u1*(u - 1)*(u + 1)/2 + (u + u_{tt})/2

h^2 =>


...

h^4 =>


\mu**3*u*u1**2*(u - 1)**2*(u + 1)**2/8 + \mu**2*u1*(8*u**4 - 9*u**2*u1**2 - 8*u**2 + 5*u1**2)/48 + \mu*u*(u**2 - 3*u1**2)/24 + u_{ttttt}/120

h^5 =>


-\mu**4*u*u1**2*(u - 1)**3*(u + 1)**3/48 - \mu**3*u1*(u - 1)*(u + 1)*(4*u**4 - 13*u**2*u1**2 - 4*u**2 + 5*u1**2)/96 - \mu**2*u*(u**4 - 9*u**2*u1**2 - u**2 + 2*u1**4 + 5*u1**2)/48 + \mu*u1*(5*u**2 - u1**2)/96 + u_{tttttt}/720

- u_{1} + u_{t}
+h\left(
\frac{\mu u_{1} \left(u - 1\right) \left(u + 1\right)}{2} + \frac{u + u_{tt}}{2}
\right)
+h^2\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{6} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{6} + \frac{u_{1} + u_{ttt}}{6}
\right)
+h^3\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{24} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{24} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{12} - \frac{u - u_{tttt}}{24}
\right)
+h^4\left(
\frac{\mu^{3} u u_{1}^{2} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{8} + \frac{\mu^{2} u_{1} \left(8 u^{4} - 9 u^{2} u_{1}^{2} - 8 u^{2} + 5 u_{1}^{2}\right)}{48} + \frac{\mu u \left(u^{2} - 3 u_{1}^{2}\right)}{24} + \frac{u_{ttttt}}{120}
\right)
+h^5\left(
- \frac{\mu^{4} u u_{1}^{2} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{48} - \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(4 u^{4} - 13 u^{2} 

In [24]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/2 - \mu*u*(u**2 - 2*u1**2 - 1)/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^4 =>


-5*\mu**4*u*u1**2*(u - 1)**3*(u + 1)**3/12 - \mu**3*u1*(u - 1)*(u + 1)*(32*u**4 - 83*u**2*u1**2 - 32*u**2 + 27*u1**2)/48 - \mu**2*u*(12*u**4 - 105*u**2*u1**2 - 12*u**2 + 26*u1**4 + 61*u1**2)/48 + \mu*u1*(29*u**2 - 10*u1**2)/48 + u1_{ttttt}/120

h^5 =>


5*\mu**5*u*u1**2*(u - 1)**4*(u + 1)**4/48 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(5*u**4 - 30*u**2*u1**2 - 5*u**2 + 9*u1**2)/24 + \mu**3*u*(10*u**6 - 209*u**4*u1**2 - 20*u**4 + 144*u**2*u1**4 + 298*u**2*u1**2 + 10*u**2 - 112*u1**4 - 89*u1**2)/96 - \mu**2*u1*(98*u**4 - 152*u**2*u1**2 - 62*u**2 + 9*u1**4 + 32*u1**2)/96 - \mu*u*(9*u**2 - 31*u1**2)/96 + u1_{tttttt}/720

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{2} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{2} + \frac{u_{1} + u1_{tt}}{2}
\right)
+h^2\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{6} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{6} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{3} - \frac{u - u1_{ttt}}{6}
\right)
+h^3\left(
- \frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{24} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{24} + \frac{\mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{24} + \frac{\mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{12} - \frac{u_{1} - u1_{tttt}}{24}
\right)
+h^4\left(
- \frac{5 \mu^{4} u u_{1}^{2} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{12} - \frac{\mu^{3} 

In [25]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


\mu**4*u1*(u - 1)**4*(u + 1)**4/120 + \mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 7*u1**2 - 1)/120 - \mu**2*u1*(6*u**4 - 5*u**2*u1**2 - 8*u**2 + u1**2 + 2)/80 - \mu*u*(3*u**2 - 7*u1**2 - 2)/120 + u1/120

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/144 + \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 7*u1**2 - 1)/144 - \mu**3*u1*(u - 1)*(u + 1)*(16*u**4 - 19*u**2*u1**2 - 24*u**2 - u1**2 + 8)/288 - \mu**2*u*(2*u**4 - 13*u**2*u1**2 - 5*u**2 + 9*u1**2 + 3)/144 + \mu*u1*(12*u**2 + u1**2 - 6)/288 + u/144

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{120} + \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 7 u_{1}^{2} - 1\right)}{120} - \frac{\mu^{2} u_{1} \left(6 u^{4} - 5 u^{2} u_{1}^{2} - 8 u^{2} + u_{1}^{2} + 2\right)}{80} - \frac{\mu u \left(3 u^{2} - 7 u_{1}^{2} - 2\right)}{120} + \frac{u_{1}}{120}
\right)
+h^5\left(
\frac{\mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{144} + \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 7 u_{1}^{2} - 1\right)}{144} - \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(16 u^{4} - 19 u^{2} u_{1}^{2} - 24 u^{2} - u_{1}^{2} + 8\right)}{288} - \frac{\mu^{2} u \left(2 u^{4} - 13 u^{2} u_{1}^{2} - 5 u^{2} + 9 u_{1}^{2} + 3\right)}{144} + \frac{\mu u_{1} \left(12 u^{2} + u_{1}^{2} - 6\right)}{288} + \frac{u}{144}
\right)


In [26]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-\mu**5*u1*(u - 1)**5*(u + 1)**5/120 - \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 2*u1**2 - 1)/120 + \mu**3*u1*(u - 1)*(u + 1)*(15*u**2*u1**2 - 8*u**2 - 7*u1**2 + 8)/240 - \mu**2*u*(2*u**4 - 25*u**2*u1**2 + 4*u**2 + 10*u1**4 + 5*u1**2 - 6)/240 + \mu*u1*(7*u**2 - 6*u1**2 + 6)/240 - u/120

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/144 - \mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 3*u1**2 - 1)/144 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(u**4 + 14*u**2*u1**2 - 6*u**2 - 2*u1**2 + 5)/144 - \mu**3*u*(2*u**6 - 47*u**4*u1**2 + 4*u**4 + 32*u**2*u1**4 + 34*u**2*u1**2 - 14*u**2 - 24*u1**4 + 13*u1**2 + 8)/288 + \mu**2*u1*(6*u**4 - 8*u**2*u1**2 + 6*u**2 + u1**4 + 2*u1**2 - 4)/96 + \mu*u*(u1**2 + 2)/96 + u1/144

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{\mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{120} - \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 2 u_{1}^{2} - 1\right)}{120} + \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(15 u^{2} u_{1}^{2} - 8 u^{2} - 7 u_{1}^{2} + 8\right)}{240} - \frac{\mu^{2} u \left(2 u^{4} - 25 u^{2} u_{1}^{2} + 4 u^{2} + 10 u_{1}^{4} + 5 u_{1}^{2} - 6\right)}{240} + \frac{\mu u_{1} \left(7 u^{2} - 6 u_{1}^{2} + 6\right)}{240} - \frac{u}{120}
\right)
+h^5\left(
- \frac{\mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{144} - \frac{\mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(u^{2} - 3 u_{1}^{2} - 1\right)}{144} + \frac{\mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{4} + 14 u^{2} u_{1}^{2} - 6 u^{2} - 2 u_{1}^{2} + 5\right)}{144} - \frac{\mu^{3} u \left(2 u^{6

In [27]:
replace(str(f2[4].collect(mu)))

'mu**5*(-y[0]**10*y[1]/120 + y[0]**8*y[1]/24 - y[0]**6*y[1]/12 + y[0]**4*y[1]/12 - y[0]**2*y[1]/24 + y[1]/120) + mu**4*(-y[0]**9/120 + y[0]**7*y[1]**2/60 + y[0]**7/30 - y[0]**5*y[1]**2/20 - y[0]**5/20 + y[0]**3*y[1]**2/20 + y[0]**3/30 - y[0]*y[1]**2/60 - y[0]/120) + mu**3*(y[0]**4*y[1]**3/16 - y[0]**4*y[1]/30 - 11*y[0]**2*y[1]**3/120 + y[0]**2*y[1]/15 + 7*y[1]**3/240 - y[1]/30) + mu**2*(-y[0]**5/120 + 5*y[0]**3*y[1]**2/48 - y[0]**3/60 - y[0]*y[1]**4/24 - y[0]*y[1]**2/48 + y[0]/40) + mu*(7*y[0]**2*y[1]/240 - y[1]**3/40 + y[1]/40) - y[0]/120'

## Dormand–Prince method 5 order

In [28]:
set_clip(8, 7, Rational(0, 1))
r = DormandPrince5(f, Matrix([u(t), u1(t)]))

In [29]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


\mu*u1*(u - 1)*(u + 1)/2 + (u + u_{tt})/2

h^2 =>


...

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/600 + \mu**4*u*(u - 1)**3*(u + 1)**3*(18*u**2 - 805*u1**2 - 18)/10800 - \mu**3*u1*(u - 1)*(u + 1)*(7392*u**4 - 18265*u**2*u1**2 - 7824*u**2 + 5781*u1**2 + 432)/64800 - \mu**2*u*(1335*u**4 - 11429*u**2*u1**2 - 1497*u**2 + 2710*u1**4 + 6879*u1**2 + 162)/32400 + \mu*u1*(2119*u**2 - 662*u1**2 - 108)/21600 + (6*u + 5*u_{tttttt})/3600

h^6 =>


7589*\mu**5*u*u1**2*(u - 1)**4*(u + 1)**4/324000 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(115326*u**4 - 583403*u**2*u1**2 - 115326*u**2 + 141093*u1**2)/2916000 + \mu**3*u*(94050*u**6 - 1877597*u**4*u1**2 - 188100*u**4 + 1221492*u**2*u1**4 + 2647382*u**2*u1**2 + 94050*u**2 - 901524*u1**4 - 769785*u1**2)/5832000 - \mu**2*u1*(207253*u**4 - 302289*u**2*u1**2 - 132123*u**2 + 17567*u1**4 + 62391*u1**2)/1458000 - \mu*u*(25871*u**2 - 73630*u1**2)/1944000 + u_{ttttttt}/5040

- u_{1} + u_{t}
+h\left(
\frac{\mu u_{1} \left(u - 1\right) \left(u + 1\right)}{2} + \frac{u + u_{tt}}{2}
\right)
+h^2\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{6} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{6} + \frac{u_{1} + u_{ttt}}{6}
\right)
+h^3\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{24} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{24} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{12} - \frac{u - u_{tttt}}{24}
\right)
+h^4\left(
- \frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{120} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{120} + \frac{\mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{120} + \frac{\mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{60} - \frac{u_{1} - u_{ttttt}}{120}
\right)
+h^5\left(
\frac{\mu^{5} u_{1} \left(u - 1\ri

In [30]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/2 - \mu*u*(u**2 - 2*u1**2 - 1)/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/600 - \mu**5*u*(u - 1)**4*(u + 1)**4*(18*u**2 - 1729*u1**2 - 18)/10800 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(4377*u**4 - 21707*u**2*u1**2 - 4512*u**2 + 5484*u1**2 + 135)/16200 + \mu**3*u*(7242*u**6 - 142305*u**4*u1**2 - 14916*u**4 + 93452*u**2*u1**4 + 202662*u**2*u1**2 + 8106*u**2 - 68972*u1**4 - 60357*u1**2 - 432)/64800 - \mu**2*u1*(32165*u**4 - 47472*u**2*u1**2 - 21189*u**2 + 2698*u1**4 + 9684*u1**2 + 324)/32400 - \mu*u*(2075*u**2 - 6130*u1**2 - 108)/21600 + (6*u1 + 5*u1_{tttttt})/3600

h^6 =>


-119239*\mu**6*u*u1**2*(u - 1)**5*(u + 1)**5/3564000 - \mu**5*u1*(u - 1)**3*(u + 1)**3*(7221384*u**4 - 88553803*u**2*u1**2 - 7221384*u**2 + 20501331*u1**2)/128304000 - \mu**4*u*(u - 1)*(u + 1)*(2928780*u**6 - 171517871*u**4*u1**2 - 5857560*u**4 + 234474782*u**2*u1**4 + 227007062*u**2*u1**2 + 2928780*u**2 - 148281182*u1**4 - 55489191*u1**2)/128304000 + \mu**3*u1*(100579861*u**6 - 358978597*u**4*u1**2 - 144206986*u**4 + 83749440*u**2*u1**4 + 323836246*u**2*u1**2 + 43627125*u**2 - 27203616*u1**4 - 31544433*u1**2)/128304000 + \mu**2*u*(16294737*u**4 - 150285356*u**2*u1**2 - 9874785*u**2 + 58326592*u1**4 + 44616750*u1**2)/128304000 - \mu*u1*(4582115*u**2 - 1493508*u1**2)/42768000 + u1_{ttttttt}/5040

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{2} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{2} + \frac{u_{1} + u1_{tt}}{2}
\right)
+h^2\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{6} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{6} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{3} - \frac{u - u1_{ttt}}{6}
\right)
+h^3\left(
- \frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{24} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{24} + \frac{\mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{24} + \frac{\mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{12} - \frac{u_{1} - u1_{tttt}}{24}
\right)
+h^4\left(
\frac{\mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{120} + \frac{\mu^{4} u \left(u

In [31]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/3600 + \mu**4*u*(u - 1)**3*(u + 1)**3*(3*u**2 - 25*u1**2 - 3)/10800 - \mu**3*u1*(u - 1)*(u + 1)*(192*u**4 - 265*u**2*u1**2 - 264*u**2 + 21*u1**2 + 72)/64800 - \mu**2*u*(30*u**4 - 179*u**2*u1**2 - 57*u**2 + 10*u1**4 + 129*u1**2 + 27)/32400 + \mu*u1*(49*u**2 - 2*u1**2 - 18)/21600 + u/3600

h^6 =>


\mu**6*u1*(u - 1)**6*(u + 1)**6/2100 + \mu**5*u*(u - 1)**4*(u + 1)**4*(270*u**2 - 2143*u1**2 - 270)/567000 - \mu**4*u1*(u - 1)**2*(u + 1)**2*(198216*u**4 - 243353*u**2*u1**2 - 295416*u**2 + 60633*u1**2 + 97200)/40824000 - \mu**3*u*(63360*u**6 - 526741*u**4*u1**2 - 204480*u**4 + 186036*u**2*u1**4 + 1017766*u**2*u1**2 + 218880*u**2 - 68892*u1**4 - 491025*u1**2 - 77760)/40824000 + \mu**2*u1*(368711*u**4 - 428448*u**2*u1**2 - 506211*u**2 - 836*u1**4 + 66912*u1**2 + 116640)/40824000 + \mu*u*(22213*u**2 - 71300*u1**2 - 19440)/13608000 - u1/2100

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{\mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{3600} + \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(3 u^{2} - 25 u_{1}^{2} - 3\right)}{10800} - \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(192 u^{4} - 265 u^{2} u_{1}^{2} - 264 u^{2} + 21 u_{1}^{2} + 72\right)}{64800} - \frac{\mu^{2} u \left(30 u^{4} - 179 u^{2} u_{1}^{2} - 57 u^{2} + 10 u_{1}^{4} + 129 u_{1}^{2} + 27\right)}{32400} + \frac{\mu u_{1} \left(49 u^{2} - 2 u_{1}^{2} - 18\right)}{21600} + \frac{u}{3600}
\right)
+h^6\left(
\frac{\mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{2100} + \frac{\mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(270 u^{2} - 2143 u_{1}^{2} - 270\right)}{567000} - \frac{\mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(198216 u^{4} - 243353 u^{2} u_{1}^{2} - 295416 u^{2} + 60633 u_{1}^{2} + 97

In [32]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/3600 - \mu**5*u*(u - 1)**4*(u + 1)**4*(3*u**2 - 19*u1**2 - 3)/10800 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(69*u**4 - 34*u**2*u1**2 - 114*u**2 - 12*u1**2 + 45)/32400 + \mu**3*u*(u - 1)*(u + 1)*(42*u**4 - 105*u**2*u1**2 - 114*u**2 - 148*u1**4 + 237*u1**2 + 72)/64800 - \mu**2*u1*(35*u**4 + 48*u**2*u1**2 - 129*u**2 - 2*u1**4 - 36*u1**2 + 54)/32400 - \mu*u*(5*u**2 - 10*u1**2 - 18)/21600 + u1/3600

h^6 =>


-\mu**7*u1*(u - 1)**7*(u + 1)**7/2100 - \mu**6*u*(u - 1)**5*(u + 1)**5*(1485*u**2 - 58459*u1**2 - 1485)/3118500 + \mu**5*u1*(u - 1)**3*(u + 1)**3*(31671792*u**4 - 89385119*u**2*u1**2 - 34237872*u**2 + 7372623*u1**2 + 2566080)/898128000 + \mu**4*u*(u - 1)*(u + 1)*(15263280*u**6 - 167930113*u**4*u1**2 - 32664960*u**4 + 132069466*u**2*u1**4 + 234275146*u**2*u1**2 + 19540080*u**2 - 22394986*u1**4 - 66345033*u1**2 - 2138400)/898128000 - \mu**3*u1*(92704853*u**6 - 204452321*u**4*u1**2 - 167926418*u**4 + 20231880*u**2*u1**4 + 166277198*u**2*u1**2 + 79498365*u**2 + 1299672*u1**4 - 13492269*u1**2 - 4276800)/898128000 - \mu**2*u*(14176491*u**4 - 99271528*u**2*u1**2 - 17330235*u**2 + 20284856*u1**4 + 47896230*u1**2 + 2566080)/898128000 + \mu*u1*(5264035*u**2 - 1746204*u1**2 - 570240)/299376000 + u/2100

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{\mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{3600} - \frac{\mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(3 u^{2} - 19 u_{1}^{2} - 3\right)}{10800} + \frac{\mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(69 u^{4} - 34 u^{2} u_{1}^{2} - 114 u^{2} - 12 u_{1}^{2} + 45\right)}{32400} + \frac{\mu^{3} u \left(u - 1\right) \left(u + 1\right) \left(42 u^{4} - 105 u^{2} u_{1}^{2} - 114 u^{2} - 148 u_{1}^{4} + 237 u_{1}^{2} + 72\right)}{64800} - \frac{\mu^{2} u_{1} \left(35 u^{4} + 48 u^{2} u_{1}^{2} - 129 u^{2} - 2 u_{1}^{4} - 36 u_{1}^{2} + 54\right)}{32400} - \frac{\mu u \left(5 u^{2} - 10 u_{1}^{2} - 18\right)}{21600} + \frac{u_{1}}{3600}
\right)
+h^6\left(
- \frac{\mu^{7} u_{1} \left(u - 1\right)^{7} \left(u + 1\right)^{7}}{2100} - \frac{\mu^{6} u \left(u - 1\right)^{5} \left(u + 1\r

In [33]:
replace(str(f2[5].collect(mu)))

'mu**6*(-y[0]**12*y[1]/3600 + y[0]**10*y[1]/600 - y[0]**8*y[1]/240 + y[0]**6*y[1]/180 - y[0]**4*y[1]/240 + y[0]**2*y[1]/600 - y[1]/3600) + mu**5*(-y[0]**11/3600 + 19*y[0]**9*y[1]**2/10800 + y[0]**9/720 - 19*y[0]**7*y[1]**2/2700 - y[0]**7/360 + 19*y[0]**5*y[1]**2/1800 + y[0]**5/360 - 19*y[0]**3*y[1]**2/2700 - y[0]**3/720 + 19*y[0]*y[1]**2/10800 + y[0]/3600) + mu**4*(23*y[0]**8*y[1]/10800 - 17*y[0]**6*y[1]**3/16200 - 7*y[0]**6*y[1]/900 + 7*y[0]**4*y[1]**3/4050 + 19*y[0]**4*y[1]/1800 - y[0]**2*y[1]**3/3240 - 17*y[0]**2*y[1]/2700 - y[1]**3/2700 + y[1]/720) + mu**3*(7*y[0]**7/10800 - 7*y[0]**5*y[1]**2/4320 - 13*y[0]**5/5400 - 37*y[0]**3*y[1]**4/16200 + 19*y[0]**3*y[1]**2/3600 + 31*y[0]**3/10800 + 37*y[0]*y[1]**4/16200 - 79*y[0]*y[1]**2/21600 - y[0]/900) + mu**2*(-7*y[0]**4*y[1]/6480 - y[0]**2*y[1]**3/675 + 43*y[0]**2*y[1]/10800 + y[1]**5/16200 + y[1]**3/900 - y[1]/600) + mu*(-y[0]**3/4320 + y[0]*y[1]**2/2160 + y[0]/1200) + y[1]/3600'

## Dormand–Prince method 4 order

In [34]:
set_clip(7, 6, Rational(0, 1))
r = DormandPrince4(f, Matrix([u(t), u1(t)]))

In [35]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


\mu*u1*(u - 1)*(u + 1)/2 + (u + u_{tt})/2

h^2 =>


...

h^4 =>


-1097*\mu**4*u1*(u - 1)**4*(u + 1)**4/120000 - \mu**3*u*(u - 1)**2*(u + 1)**2*(3291*u**2 - 68338*u1**2 - 3291)/360000 + \mu**2*u1*(89851*u**4 - 92126*u**2*u1**2 - 99724*u**2 + 42518*u1**2 + 9873)/360000 + \mu*u*(4134*u**2 - 11305*u1**2 - 1097)/60000 - (1097*u1 - 1000*u_{ttttt})/120000

h^5 =>


161*\mu**5*u1*(u - 1)**5*(u + 1)**5/120000 + \mu**4*u*(u - 1)**3*(u + 1)**3*(7245*u**2 - 414181*u1**2 - 7245)/5400000 - \mu**3*u1*(u - 1)*(u + 1)*(3832068*u**4 - 9628561*u**2*u1**2 - 4005948*u**2 + 3022005*u1**2 + 173880)/32400000 - \mu**2*u*(695226*u**4 - 6009689*u**2*u1**2 - 760431*u**2 + 1454902*u1**4 + 3576639*u1**2 + 65205)/16200000 + \mu*u1*(1102681*u**2 - 359702*u1**2 - 43470)/10800000 + (483*u + 500*u_{tttttt})/360000

- u_{1} + u_{t}
+h\left(
\frac{\mu u_{1} \left(u - 1\right) \left(u + 1\right)}{2} + \frac{u + u_{tt}}{2}
\right)
+h^2\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{6} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{6} + \frac{u_{1} + u_{ttt}}{6}
\right)
+h^3\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{24} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{24} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{12} - \frac{u - u_{tttt}}{24}
\right)
+h^4\left(
- \frac{1097 \mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{120000} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(3291 u^{2} - 68338 u_{1}^{2} - 3291\right)}{360000} + \frac{\mu^{2} u_{1} \left(89851 u^{4} - 92126 u^{2} u_{1}^{2} - 99724 u^{2} + 42518 u_{1}^{2} + 9873\right)}{360000} + \frac{\mu u \left(4134 u^{2} - 11305 u_{1}^{2} - 1097\right)}{60000} - \frac{1097 u_{1} - 1000 u_{tttt

In [36]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/2 - \mu*u*(u**2 - 2*u1**2 - 1)/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^4 =>


1097*\mu**5*u1*(u - 1)**5*(u + 1)**5/120000 + \mu**4*u*(u - 1)**3*(u + 1)**3*(9873*u**2 - 473210*u1**2 - 9873)/1080000 - \mu**3*u1*(u - 1)*(u + 1)*(2177328*u**4 - 5403487*u**2*u1**2 - 2295804*u**2 + 1726059*u1**2 + 118476)/3240000 - \mu**2*u*(787317*u**4 - 6756028*u**2*u1**2 - 876174*u**2 + 1609688*u1**4 + 4065324*u1**2 + 88857)/3240000 + \mu*u1*(623650*u**2 - 196376*u1**2 - 29619)/1080000 + (1097*u + 1000*u1_{ttttt})/120000

h^5 =>


-161*\mu**6*u1*(u - 1)**6*(u + 1)**6/120000 - 7*\mu**5*u*(u - 1)**4*(u + 1)**4*(621*u**2 - 75307*u1**2 - 621)/3240000 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(39869937*u**4 - 198193667*u**2*u1**2 - 40848012*u**2 + 49243833*u1**2 + 978075)/145800000 + \mu**3*u*(32687694*u**6 - 645389207*u**4*u1**2 - 66940308*u**4 + 421801644*u**2*u1**4 + 916591154*u**2*u1**2 + 35817534*u**2 - 310930284*u1**4 - 271201947*u1**2 - 1564920)/291600000 - \mu**2*u1*(72403201*u**4 - 106255365*u**2*u1**2 - 47354502*u**2 + 6083927*u1**4 + 21671787*u1**2 + 586845)/72900000 - \mu*u*(9281039*u**2 - 26983666*u1**2 - 391230)/97200000 + (483*u1 + 500*u1_{tttttt})/360000

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
- \frac{\mu^{2} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2}}{2} - \frac{\mu u \left(u^{2} - 2 u_{1}^{2} - 1\right)}{2} + \frac{u_{1} + u1_{tt}}{2}
\right)
+h^2\left(
\frac{\mu^{3} u_{1} \left(u - 1\right)^{3} \left(u + 1\right)^{3}}{6} + \frac{\mu^{2} u \left(u - 1\right) \left(u + 1\right) \left(u^{2} - 8 u_{1}^{2} - 1\right)}{6} - \frac{\mu u_{1} \left(4 u^{2} - u_{1}^{2} - 1\right)}{3} - \frac{u - u1_{ttt}}{6}
\right)
+h^3\left(
- \frac{\mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{24} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{24} + \frac{\mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{24} + \frac{\mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{12} - \frac{u_{1} - u1_{tttt}}{24}
\right)
+h^4\left(
\frac{1097 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{120000} + \frac{\mu^{4} u

In [37]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-97*\mu**4*u1*(u - 1)**4*(u + 1)**4/120000 - \mu**3*u*(u - 1)**2*(u + 1)**2*(291*u**2 - 2338*u1**2 - 291)/360000 + \mu**2*u1*(2851*u**4 - 2126*u**2*u1**2 - 3724*u**2 + 518*u1**2 + 873)/360000 + \mu*u*(134*u**2 - 305*u1**2 - 97)/60000 - 97*u1/120000

h^5 =>


-77*\mu**5*u1*(u - 1)**5*(u + 1)**5/90000 - \mu**4*u*(u - 1)**3*(u + 1)**3*(2310*u**2 - 20687*u1**2 - 2310)/2700000 + \mu**3*u1*(u - 1)*(u + 1)*(142476*u**4 - 100967*u**2*u1**2 - 197916*u**2 + 11715*u1**2 + 55440)/16200000 + \mu**2*u*(16107*u**4 - 56413*u**2*u1**2 - 36897*u**2 - 17506*u1**4 + 60783*u1**2 + 20790)/8100000 - \mu*u1*(20507*u**2 + 5186*u1**2 - 13860)/5400000 - 77*u/90000

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{97 \mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{120000} - \frac{\mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(291 u^{2} - 2338 u_{1}^{2} - 291\right)}{360000} + \frac{\mu^{2} u_{1} \left(2851 u^{4} - 2126 u^{2} u_{1}^{2} - 3724 u^{2} + 518 u_{1}^{2} + 873\right)}{360000} + \frac{\mu u \left(134 u^{2} - 305 u_{1}^{2} - 97\right)}{60000} - \frac{97 u_{1}}{120000}
\right)
+h^5\left(
- \frac{77 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{90000} - \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(2310 u^{2} - 20687 u_{1}^{2} - 2310\right)}{2700000} + \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(142476 u^{4} - 100967 u^{2} u_{1}^{2} - 197916 u^{2} + 11715 u_{1}^{2} + 55440\right)}{16200000} + \frac{\mu^{2} u \left(16107 u^{4} - 56413 u^{2} u_{1}^{2} - 36897 u^{2} - 17506 u_{1}^{4} + 60783 u_{1}^{2} + 20790\right)

In [38]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


97*\mu**5*u1*(u - 1)**5*(u + 1)**5/120000 + \mu**4*u*(u - 1)**3*(u + 1)**3*(873*u**2 - 5210*u1**2 - 873)/1080000 - \mu**3*u1*(u - 1)*(u + 1)*(17328*u**4 - 3487*u**2*u1**2 - 27804*u**2 - 1941*u1**2 + 10476)/3240000 - \mu**2*u*(4317*u**4 - 6028*u**2*u1**2 - 12174*u**2 - 10312*u1**4 + 15324*u1**2 + 7857)/3240000 + \mu*u1*(2650*u**2 + 1624*u1**2 - 2619)/1080000 + 97*u/120000

h^5 =>


77*\mu**6*u1*(u - 1)**6*(u + 1)**6/90000 + \mu**5*u*(u - 1)**4*(u + 1)**4*(1386*u**2 - 12505*u1**2 - 1386)/1620000 - \mu**4*u1*(u - 1)**2*(u + 1)**2*(1402803*u**4 - 738778*u**2*u1**2 - 2026503*u**2 + 343152*u1**2 + 623700)/145800000 - \mu**3*u*(402093*u**6 - 1865039*u**4*u1**2 - 1303146*u**4 - 286062*u**2*u1**4 + 4987028*u**2*u1**2 + 1400013*u**2 + 825882*u1**4 - 3121989*u1**2 - 498960)/145800000 + \mu**2*u1*(1796218*u**4 - 1541265*u**2*u1**2 - 3273456*u**2 - 249874*u1**4 + 407631*u1**2 + 748440)/145800000 + \mu*u*(150518*u**2 - 450607*u1**2 - 124740)/48600000 - 77*u1/90000

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{97 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{120000} + \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(873 u^{2} - 5210 u_{1}^{2} - 873\right)}{1080000} - \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(17328 u^{4} - 3487 u^{2} u_{1}^{2} - 27804 u^{2} - 1941 u_{1}^{2} + 10476\right)}{3240000} - \frac{\mu^{2} u \left(4317 u^{4} - 6028 u^{2} u_{1}^{2} - 12174 u^{2} - 10312 u_{1}^{4} + 15324 u_{1}^{2} + 7857\right)}{3240000} + \frac{\mu u_{1} \left(2650 u^{2} + 1624 u_{1}^{2} - 2619\right)}{1080000} + \frac{97 u}{120000}
\right)
+h^5\left(
\frac{77 \mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{90000} + \frac{\mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(1386 u^{2} - 12505 u_{1}^{2} - 1386\right)}{1620000} - \frac{\mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)

In [39]:
replace(str(f2[4].collect(mu)))

'mu**5*(97*y[0]**10*y[1]/120000 - 97*y[0]**8*y[1]/24000 + 97*y[0]**6*y[1]/12000 - 97*y[0]**4*y[1]/12000 + 97*y[0]**2*y[1]/24000 - 97*y[1]/120000) + mu**4*(97*y[0]**9/120000 - 521*y[0]**7*y[1]**2/108000 - 97*y[0]**7/30000 + 521*y[0]**5*y[1]**2/36000 + 97*y[0]**5/20000 - 521*y[0]**3*y[1]**2/36000 - 97*y[0]**3/30000 + 521*y[0]*y[1]**2/108000 + 97*y[0]/120000) + mu**3*(-361*y[0]**6*y[1]/67500 + 3487*y[0]**4*y[1]**3/3240000 + 3761*y[0]**4*y[1]/270000 - 773*y[0]**2*y[1]**3/1620000 - 319*y[0]**2*y[1]/27000 - 647*y[1]**3/1080000 + 97*y[1]/30000) + mu**2*(-1439*y[0]**5/1080000 + 1507*y[0]**3*y[1]**2/810000 + 2029*y[0]**3/540000 + 1289*y[0]*y[1]**4/405000 - 1277*y[0]*y[1]**2/270000 - 97*y[0]/40000) + mu*(53*y[0]**2*y[1]/21600 + 203*y[1]**3/135000 - 97*y[1]/40000) + 97*y[0]/120000'

### Adams–Bashforth method 4 order

In [40]:
set_clip(7, 6, Rational(0, 1))
r = AdamsBashforth4(g, Matrix([u(t), u1(t)]))

In [41]:
F1 = clip((T(u(t+4*h))-T(u(t+3*h)))/h - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-7*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-(4435*u1_{tttt} - 4686*u_{ttttt})/720

h^5 =>


-(5757*u1_{ttttt} - 6734*u_{tttttt})/1440

- u_{1} + u_{t}
+h\left(
- \frac{7 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{4435 u1_{tttt} - 4686 u_{ttttt}}{720}
\right)
+h^5\left(
- \frac{5757 u1_{ttttt} - 6734 u_{tttttt}}{1440}
\right)


In [42]:
F2 = clip((T(u1(t+4*h))-T(u1(t+3*h)))/h - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


7*\mu*(u**2*u1_{t} + 2*u*u1*u_{t} - u1_{t})/2 + 7*(u1_{tt} + u_{t})/2

h^2 =>


...

h^4 =>


887*\mu*(u**2*u1_{tttt} + 2*u*u1*u_{tttt} + 8*u*u1_{ttt}*u_{t} + 12*u*u1_{tt}*u_{tt} + 8*u*u1_{t}*u_{ttt} + 8*u1*u_{ttt}*u_{t} + 6*u1*u_{tt}**2 - u1_{tttt} + 12*u1_{tt}*u_{t}**2 + 24*u1_{t}*u_{tt}*u_{t})/144 + (4686*u1_{ttttt} + 4435*u_{tttt})/720

h^5 =>


1919*\mu*(u**2*u1_{ttttt} + 2*u*u1*u_{ttttt} + 10*u*u1_{tttt}*u_{t} + 20*u*u1_{ttt}*u_{tt} + 20*u*u1_{tt}*u_{ttt} + 10*u*u1_{t}*u_{tttt} + 10*u1*u_{tttt}*u_{t} + 20*u1*u_{ttt}*u_{tt} - u1_{ttttt} + 20*u1_{ttt}*u_{t}**2 + 60*u1_{tt}*u_{tt}*u_{t} + 40*u1_{t}*u_{ttt}*u_{t} + 30*u1_{t}*u_{tt}**2)/480 + (6734*u1_{tttttt} + 5757*u_{ttttt})/1440

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{7 \mu \left(u^{2} u1_{t} + 2 u u_{1} u_{t} - u1_{t}\right)}{2} + \frac{7 \left(u1_{tt} + u_{t}\right)}{2}
\right)
+h^2\left(
\frac{37 \mu \left(u^{2} u1_{tt} + 2 u u_{1} u_{tt} + 4 u u1_{t} u_{t} + 2 u_{1} u_{t}^{2} - u1_{tt}\right)}{6} + \frac{37 \left(u1_{ttt} + u_{tt}\right)}{6}
\right)
+h^3\left(
\frac{175 \mu \left(u^{2} u1_{ttt} + 2 u u_{1} u_{ttt} + 6 u u1_{tt} u_{t} + 6 u u1_{t} u_{tt} + 6 u_{1} u_{tt} u_{t} - u1_{ttt} + 6 u1_{t} u_{t}^{2}\right)}{24} + \frac{175 \left(u1_{tttt} + u_{ttt}\right)}{24}
\right)
+h^4\left(
\frac{887 \mu \left(u^{2} u1_{tttt} + 2 u u_{1} u_{tttt} + 8 u u1_{ttt} u_{t} + 12 u u1_{tt} u_{tt} + 8 u u1_{t} u_{ttt} + 8 u_{1} u_{ttt} u_{t} + 6 u_{1} u_{tt}^{2} - u1_{tttt} + 12 u1_{tt} u_{t}^{2} + 24 u1_{t} u_{tt} u_{t}\right)}{144} + \frac{4686 u1_{ttttt} + 4435 u_{tttt}}{720}
\right)
+h^5\left(
\frac{1919 \mu \left(u^{2} u1_{ttttt} + 2 u u_{1} u_{ttttt} + 10 u u1_{tttt} u_{t} + 20

In [43]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


251*\mu**4*u1*(u - 1)**4*(u + 1)**4/720 + 251*\mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 22*u1**2 - 1)/720 - 251*\mu**2*u1*(29*u**4 - 30*u**2*u1**2 - 32*u**2 + 14*u1**2 + 3)/720 - 251*\mu*u*(4*u**2 - 11*u1**2 - 1)/360 + 251*u1/720

h^5 =>


13*\mu**5*u1*(u - 1)**5*(u + 1)**5/24 + 13*\mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/24 - 13*\mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/6 - 13*\mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/24 + 13*\mu*u1*(69*u**2 - 22*u1**2 - 3)/24 + 13*u/24

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{251 \mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{720} + \frac{251 \mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{720} - \frac{251 \mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{720} - \frac{251 \mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{360} + \frac{251 u_{1}}{720}
\right)
+h^5\left(
\frac{13 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{24} + \frac{13 \mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{24} - \frac{13 \mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{6} - \frac{13 \mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{24} + \frac{13 \mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{24} + \fr

In [44]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-251*\mu**5*u1*(u - 1)**5*(u + 1)**5/720 - 251*\mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/720 + 251*\mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/180 + 251*\mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/720 - 251*\mu*u1*(69*u**2 - 22*u1**2 - 3)/720 - 251*u/720

h^5 =>


-13*\mu**6*u1*(u - 1)**6*(u + 1)**6/24 - 13*\mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 114*u1**2 - 1)/24 + 13*\mu**4*u1*(u - 1)**2*(u + 1)**2*(193*u**4 - 964*u**2*u1**2 - 198*u**2 + 244*u1**2 + 5)/24 + 13*\mu**3*u*(20*u**6 - 395*u**4*u1**2 - 41*u**4 + 260*u**2*u1**4 + 562*u**2*u1**2 + 22*u**2 - 192*u1**4 - 167*u1**2 - 1)/6 - 13*\mu**2*u1*(119*u**4 - 176*u**2*u1**2 - 78*u**2 + 10*u1**4 + 36*u1**2 + 1)/4 - 13*\mu*u*(23*u**2 - 68*u1**2 - 1)/8 + 13*u1/24

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{720} - \frac{251 \mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{720} + \frac{251 \mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{180} + \frac{251 \mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{720} - \frac{251 \mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{720} - \frac{251 u}{720}
\right)
+h^5\left(
- \frac{13 \mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{24} - \frac{13 \mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(u^{2} - 114 u_{1}^{2} - 1\right)}{24} + \frac{13 \mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(193 u^{4} - 964 u^{2} u_{1}^{2} - 198 u^{2} 

In [45]:
replace(str(f2[4].collect(mu)))

'mu**5*(-251*y[0]**10*y[1]/720 + 251*y[0]**8*y[1]/144 - 251*y[0]**6*y[1]/72 + 251*y[0]**4*y[1]/72 - 251*y[0]**2*y[1]/144 + 251*y[1]/720) + mu**4*(-251*y[0]**9/720 + 3263*y[0]**7*y[1]**2/180 + 251*y[0]**7/180 - 3263*y[0]**5*y[1]**2/60 - 251*y[0]**5/120 + 3263*y[0]**3*y[1]**2/60 + 251*y[0]**3/180 - 3263*y[0]*y[1]**2/180 - 251*y[0]/720) + mu**3*(251*y[0]**6*y[1]/9 - 1255*y[0]**4*y[1]**3/18 - 10291*y[0]**4*y[1]/180 + 2761*y[0]**2*y[1]**3/30 + 2761*y[0]**2*y[1]/90 - 1004*y[1]**3/45 - 251*y[1]/180) + mu**2*(7279*y[0]**5/720 - 6275*y[0]**3*y[1]**2/72 - 502*y[0]**3/45 + 251*y[0]*y[1]**4/12 + 1255*y[0]*y[1]**2/24 + 251*y[0]/240) + mu*(-5773*y[0]**2*y[1]/240 + 2761*y[1]**3/360 + 251*y[1]/240) - 251*y[0]/720'

### Adams–Moulton method 4 order

In [46]:
set_clip(7, 6, Rational(0, 1))
r = AdamsMoulton4(g, Matrix([u(t), u1(t)]))

In [47]:
F1 = clip((T(u(t+3*h))-T(u(t+2*h)))/h - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-5*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-(1285*u1_{tttt} - 1266*u_{ttttt})/720

h^5 =>


-(279*u1_{ttttt} - 266*u_{tttttt})/288

- u_{1} + u_{t}
+h\left(
- \frac{5 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{19 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{65 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{1285 u1_{tttt} - 1266 u_{ttttt}}{720}
\right)
+h^5\left(
- \frac{279 u1_{ttttt} - 266 u_{tttttt}}{288}
\right)


In [48]:
F2 = clip((T(u1(t+3*h))-T(u1(t+2*h)))/h - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


5*\mu*(u**2*u1_{t} + 2*u*u1*u_{t} - u1_{t})/2 + 5*(u1_{tt} + u_{t})/2

h^2 =>


...

h^4 =>


257*\mu*(u**2*u1_{tttt} + 2*u*u1*u_{tttt} + 8*u*u1_{ttt}*u_{t} + 12*u*u1_{tt}*u_{tt} + 8*u*u1_{t}*u_{ttt} + 8*u1*u_{ttt}*u_{t} + 6*u1*u_{tt}**2 - u1_{tttt} + 12*u1_{tt}*u_{t}**2 + 24*u1_{t}*u_{tt}*u_{t})/144 + (1266*u1_{ttttt} + 1285*u_{tttt})/720

h^5 =>


31*\mu*(u**2*u1_{ttttt} + 2*u*u1*u_{ttttt} + 10*u*u1_{tttt}*u_{t} + 20*u*u1_{ttt}*u_{tt} + 20*u*u1_{tt}*u_{ttt} + 10*u*u1_{t}*u_{tttt} + 10*u1*u_{tttt}*u_{t} + 20*u1*u_{ttt}*u_{tt} - u1_{ttttt} + 20*u1_{ttt}*u_{t}**2 + 60*u1_{tt}*u_{tt}*u_{t} + 40*u1_{t}*u_{ttt}*u_{t} + 30*u1_{t}*u_{tt}**2)/32 + (266*u1_{tttttt} + 279*u_{ttttt})/288

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{5 \mu \left(u^{2} u1_{t} + 2 u u_{1} u_{t} - u1_{t}\right)}{2} + \frac{5 \left(u1_{tt} + u_{t}\right)}{2}
\right)
+h^2\left(
\frac{19 \mu \left(u^{2} u1_{tt} + 2 u u_{1} u_{tt} + 4 u u1_{t} u_{t} + 2 u_{1} u_{t}^{2} - u1_{tt}\right)}{6} + \frac{19 \left(u1_{ttt} + u_{tt}\right)}{6}
\right)
+h^3\left(
\frac{65 \mu \left(u^{2} u1_{ttt} + 2 u u_{1} u_{ttt} + 6 u u1_{tt} u_{t} + 6 u u1_{t} u_{tt} + 6 u_{1} u_{tt} u_{t} - u1_{ttt} + 6 u1_{t} u_{t}^{2}\right)}{24} + \frac{65 \left(u1_{tttt} + u_{ttt}\right)}{24}
\right)
+h^4\left(
\frac{257 \mu \left(u^{2} u1_{tttt} + 2 u u_{1} u_{tttt} + 8 u u1_{ttt} u_{t} + 12 u u1_{tt} u_{tt} + 8 u u1_{t} u_{ttt} + 8 u_{1} u_{ttt} u_{t} + 6 u_{1} u_{tt}^{2} - u1_{tttt} + 12 u1_{tt} u_{t}^{2} + 24 u1_{t} u_{tt} u_{t}\right)}{144} + \frac{1266 u1_{ttttt} + 1285 u_{tttt}}{720}
\right)
+h^5\left(
\frac{31 \mu \left(u^{2} u1_{ttttt} + 2 u u_{1} u_{ttttt} + 10 u u1_{tttt} u_{t} + 20 u u

In [49]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-19*\mu**4*u1*(u - 1)**4*(u + 1)**4/720 - 19*\mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 22*u1**2 - 1)/720 + 19*\mu**2*u1*(29*u**4 - 30*u**2*u1**2 - 32*u**2 + 14*u1**2 + 3)/720 + 19*\mu*u*(4*u**2 - 11*u1**2 - 1)/360 - 19*u1/720

h^5 =>


-\mu**5*u1*(u - 1)**5*(u + 1)**5/48 - \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/48 + \mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/12 + \mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/48 - \mu*u1*(69*u**2 - 22*u1**2 - 3)/48 - u/48

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{19 \mu^{4} u_{1} \left(u - 1\right)^{4} \left(u + 1\right)^{4}}{720} - \frac{19 \mu^{3} u \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(u^{2} - 22 u_{1}^{2} - 1\right)}{720} + \frac{19 \mu^{2} u_{1} \left(29 u^{4} - 30 u^{2} u_{1}^{2} - 32 u^{2} + 14 u_{1}^{2} + 3\right)}{720} + \frac{19 \mu u \left(4 u^{2} - 11 u_{1}^{2} - 1\right)}{360} - \frac{19 u_{1}}{720}
\right)
+h^5\left(
- \frac{\mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{48} - \frac{\mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{48} + \frac{\mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{12} + \frac{\mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{48} - \frac{\mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{48} - \frac{u}{48}
\righ

In [50]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


19*\mu**5*u1*(u - 1)**5*(u + 1)**5/720 + 19*\mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/720 - 19*\mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/180 - 19*\mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/720 + 19*\mu*u1*(69*u**2 - 22*u1**2 - 3)/720 + 19*u/720

h^5 =>


\mu**6*u1*(u - 1)**6*(u + 1)**6/48 + \mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 114*u1**2 - 1)/48 - \mu**4*u1*(u - 1)**2*(u + 1)**2*(193*u**4 - 964*u**2*u1**2 - 198*u**2 + 244*u1**2 + 5)/48 - \mu**3*u*(20*u**6 - 395*u**4*u1**2 - 41*u**4 + 260*u**2*u1**4 + 562*u**2*u1**2 + 22*u**2 - 192*u1**4 - 167*u1**2 - 1)/12 + \mu**2*u1*(119*u**4 - 176*u**2*u1**2 - 78*u**2 + 10*u1**4 + 36*u1**2 + 1)/8 + \mu*u*(23*u**2 - 68*u1**2 - 1)/16 - u1/48

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{720} + \frac{19 \mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{720} - \frac{19 \mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{180} - \frac{19 \mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{720} + \frac{19 \mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{720} + \frac{19 u}{720}
\right)
+h^5\left(
\frac{\mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{48} + \frac{\mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(u^{2} - 114 u_{1}^{2} - 1\right)}{48} - \frac{\mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(193 u^{4} - 964 u^{2} u_{1}^{2} - 198 u^{2} + 244 u_{1}^{2} + 5

In [51]:
replace(str(f2[4].collect(mu)))

'mu**5*(19*y[0]**10*y[1]/720 - 19*y[0]**8*y[1]/144 + 19*y[0]**6*y[1]/72 - 19*y[0]**4*y[1]/72 + 19*y[0]**2*y[1]/144 - 19*y[1]/720) + mu**4*(19*y[0]**9/720 - 247*y[0]**7*y[1]**2/180 - 19*y[0]**7/180 + 247*y[0]**5*y[1]**2/60 + 19*y[0]**5/120 - 247*y[0]**3*y[1]**2/60 - 19*y[0]**3/180 + 247*y[0]*y[1]**2/180 + 19*y[0]/720) + mu**3*(-19*y[0]**6*y[1]/9 + 95*y[0]**4*y[1]**3/18 + 779*y[0]**4*y[1]/180 - 209*y[0]**2*y[1]**3/30 - 209*y[0]**2*y[1]/90 + 76*y[1]**3/45 + 19*y[1]/180) + mu**2*(-551*y[0]**5/720 + 475*y[0]**3*y[1]**2/72 + 38*y[0]**3/45 - 19*y[0]*y[1]**4/12 - 95*y[0]*y[1]**2/24 - 19*y[0]/240) + mu*(437*y[0]**2*y[1]/240 - 209*y[1]**3/360 - 19*y[1]/240) + 19*y[0]/720'

### Adams–Bashforth method 5 order

In [52]:
set_clip(7, 6, Rational(0, 1))
r = AdamsBashforth5(g, Matrix([u(t), u1(t)]))

In [53]:
F1 = clip((T(u(t+5*h))-T(u(t+4*h)))/h - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-9*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-2101*(u1_{tttt} - u_{ttttt})/120

h^5 =>


-(22583*u1_{ttttt} - 23058*u_{tttttt})/1440

- u_{1} + u_{t}
+h\left(
- \frac{9 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{61 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{123 \left(u1_{ttt} - u_{tttt}\right)}{8}
\right)
+h^4\left(
- \frac{2101 \left(u1_{tttt} - u_{ttttt}\right)}{120}
\right)
+h^5\left(
- \frac{22583 u1_{ttttt} - 23058 u_{tttttt}}{1440}
\right)


In [54]:
F2 = clip((T(u1(t+5*h))-T(u1(t+4*h)))/h - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


9*\mu*(u**2*u1_{t} + 2*u*u1*u_{t} - u1_{t})/2 + 9*(u1_{tt} + u_{t})/2

h^2 =>


...

h^4 =>


2101*\mu*(u**2*u1_{tttt} + 2*u*u1*u_{tttt} + 8*u*u1_{ttt}*u_{t} + 12*u*u1_{tt}*u_{tt} + 8*u*u1_{t}*u_{ttt} + 8*u1*u_{ttt}*u_{t} + 6*u1*u_{tt}**2 - u1_{tttt} + 12*u1_{tt}*u_{t}**2 + 24*u1_{t}*u_{tt}*u_{t})/120 + 2101*(u1_{ttttt} + u_{tttt})/120

h^5 =>


22583*\mu*(u**2*u1_{ttttt} + 2*u*u1*u_{ttttt} + 10*u*u1_{tttt}*u_{t} + 20*u*u1_{ttt}*u_{tt} + 20*u*u1_{tt}*u_{ttt} + 10*u*u1_{t}*u_{tttt} + 10*u1*u_{tttt}*u_{t} + 20*u1*u_{ttt}*u_{tt} - u1_{ttttt} + 20*u1_{ttt}*u_{t}**2 + 60*u1_{tt}*u_{tt}*u_{t} + 40*u1_{t}*u_{ttt}*u_{t} + 30*u1_{t}*u_{tt}**2)/1440 + (23058*u1_{tttttt} + 22583*u_{ttttt})/1440

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{9 \mu \left(u^{2} u1_{t} + 2 u u_{1} u_{t} - u1_{t}\right)}{2} + \frac{9 \left(u1_{tt} + u_{t}\right)}{2}
\right)
+h^2\left(
\frac{61 \mu \left(u^{2} u1_{tt} + 2 u u_{1} u_{tt} + 4 u u1_{t} u_{t} + 2 u_{1} u_{t}^{2} - u1_{tt}\right)}{6} + \frac{61 \left(u1_{ttt} + u_{tt}\right)}{6}
\right)
+h^3\left(
\frac{123 \mu \left(u^{2} u1_{ttt} + 2 u u_{1} u_{ttt} + 6 u u1_{tt} u_{t} + 6 u u1_{t} u_{tt} + 6 u_{1} u_{tt} u_{t} - u1_{ttt} + 6 u1_{t} u_{t}^{2}\right)}{8} + \frac{123 \left(u1_{tttt} + u_{ttt}\right)}{8}
\right)
+h^4\left(
\frac{2101 \mu \left(u^{2} u1_{tttt} + 2 u u_{1} u_{tttt} + 8 u u1_{ttt} u_{t} + 12 u u1_{tt} u_{tt} + 8 u u1_{t} u_{ttt} + 8 u_{1} u_{ttt} u_{t} + 6 u_{1} u_{tt}^{2} - u1_{tttt} + 12 u1_{tt} u_{t}^{2} + 24 u1_{t} u_{tt} u_{t}\right)}{120} + \frac{2101 \left(u1_{ttttt} + u_{tttt}\right)}{120}
\right)
+h^5\left(
\frac{22583 \mu \left(u^{2} u1_{ttttt} + 2 u u_{1} u_{ttttt} + 10 u u1_{tttt} u_

In [55]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-95*\mu**5*u1*(u - 1)**5*(u + 1)**5/288 - 95*\mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/288 + 95*\mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/72 + 95*\mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/288 - 95*\mu*u1*(69*u**2 - 22*u1**2 - 3)/288 - 95*u/288

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{95 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{288} - \frac{95 \mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{288} + \frac{95 \mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{72} + \frac{95 \mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{288} - \frac{95 \mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{288} - \frac{95 u}{288}
\right)


In [56]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


95*\mu**6*u1*(u - 1)**6*(u + 1)**6/288 + 95*\mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 114*u1**2 - 1)/288 - 95*\mu**4*u1*(u - 1)**2*(u + 1)**2*(193*u**4 - 964*u**2*u1**2 - 198*u**2 + 244*u1**2 + 5)/288 - 95*\mu**3*u*(20*u**6 - 395*u**4*u1**2 - 41*u**4 + 260*u**2*u1**4 + 562*u**2*u1**2 + 22*u**2 - 192*u1**4 - 167*u1**2 - 1)/72 + 95*\mu**2*u1*(119*u**4 - 176*u**2*u1**2 - 78*u**2 + 10*u1**4 + 36*u1**2 + 1)/48 + 95*\mu*u*(23*u**2 - 68*u1**2 - 1)/96 - 95*u1/288

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{95 \mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{288} + \frac{95 \mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(u^{2} - 114 u_{1}^{2} - 1\right)}{288} - \frac{95 \mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(193 u^{4} - 964 u^{2} u_{1}^{2} - 198 u^{2} + 244 u_{1}^{2} + 5\right)}{288} - \frac{95 \mu^{3} u \left(20 u^{6} - 395 u^{4} u_{1}^{2} - 41 u^{4} + 260 u^{2} u_{1}^{4} + 562 u^{2} u_{1}^{2} + 22 u^{2} - 192 u_{1}^{4} - 167 u_{1}^{2} - 1\right)}{72} + \frac{95 \mu^{2} u_{1} \left(119 u^{4} - 176 u^{2} u_{1}^{2} - 78 u^{2} + 10 u_{1}^{4} + 36 u_{1}^{2} + 1\right)}{48} + \frac{95 \mu u \left(23 u^{2} - 68 u_{1}^{2} - 1\right)}{96} - \frac{95 u_{1}}{288}
\right)


In [57]:
replace(str(f2[5].collect(mu)))

'mu**6*(95*y[0]**12*y[1]/288 - 95*y[0]**10*y[1]/48 + 475*y[0]**8*y[1]/96 - 475*y[0]**6*y[1]/72 + 475*y[0]**4*y[1]/96 - 95*y[0]**2*y[1]/48 + 95*y[1]/288) + mu**5*(95*y[0]**11/288 - 1805*y[0]**9*y[1]**2/48 - 475*y[0]**9/288 + 1805*y[0]**7*y[1]**2/12 + 475*y[0]**7/144 - 1805*y[0]**5*y[1]**2/8 - 475*y[0]**5/144 + 1805*y[0]**3*y[1]**2/12 + 475*y[0]**3/288 - 1805*y[0]*y[1]**2/48 - 95*y[0]/288) + mu**4*(-18335*y[0]**8*y[1]/288 + 22895*y[0]**6*y[1]**3/72 + 6935*y[0]**6*y[1]/36 - 17195*y[0]**4*y[1]**3/24 - 3135*y[0]**4*y[1]/16 + 11495*y[0]**2*y[1]**3/24 + 1235*y[0]**2*y[1]/18 - 5795*y[1]**3/72 - 475*y[1]/288) + mu**3*(-475*y[0]**7/18 + 37525*y[0]**5*y[1]**2/72 + 3895*y[0]**5/72 - 6175*y[0]**3*y[1]**4/18 - 26695*y[0]**3*y[1]**2/36 - 1045*y[0]**3/36 + 760*y[0]*y[1]**4/3 + 15865*y[0]*y[1]**2/72 + 95*y[0]/72) + mu**2*(11305*y[0]**4*y[1]/48 - 1045*y[0]**2*y[1]**3/3 - 1235*y[0]**2*y[1]/8 + 475*y[1]**5/24 + 285*y[1]**3/4 + 95*y[1]/48) + mu*(2185*y[0]**3/96 - 1615*y[0]*y[1]**2/24 - 95*y[0]/96) - 95*y[1

### Adams–Moulton method 5 order

In [58]:
set_clip(7, 6, Rational(0, 1))
r = AdamsMoulton5(g, Matrix([u(t), u1(t)]))

In [59]:
F1 = clip((T(u(t+4*h))-T(u(t+3*h)))/h - T(r[0]))
prn(F1, mu)
prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-7*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-781*(u1_{tttt} - u_{ttttt})/120

h^5 =>


-(6761*u1_{ttttt} - 6734*u_{tttttt})/1440

- u_{1} + u_{t}
+h\left(
- \frac{7 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{781 \left(u1_{tttt} - u_{ttttt}\right)}{120}
\right)
+h^5\left(
- \frac{6761 u1_{ttttt} - 6734 u_{tttttt}}{1440}
\right)


In [60]:
F2 = clip((T(u1(t+4*h))-T(u1(t+3*h)))/h - T(r[1]))
prn(F2, mu)
prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


7*\mu*(u**2*u1_{t} + 2*u*u1*u_{t} - u1_{t})/2 + 7*(u1_{tt} + u_{t})/2

h^2 =>


...

h^4 =>


781*\mu*(u**2*u1_{tttt} + 2*u*u1*u_{tttt} + 8*u*u1_{ttt}*u_{t} + 12*u*u1_{tt}*u_{tt} + 8*u*u1_{t}*u_{ttt} + 8*u1*u_{ttt}*u_{t} + 6*u1*u_{tt}**2 - u1_{tttt} + 12*u1_{tt}*u_{t}**2 + 24*u1_{t}*u_{tt}*u_{t})/120 + 781*(u1_{ttttt} + u_{tttt})/120

h^5 =>


6761*\mu*(u**2*u1_{ttttt} + 2*u*u1*u_{ttttt} + 10*u*u1_{tttt}*u_{t} + 20*u*u1_{ttt}*u_{tt} + 20*u*u1_{tt}*u_{ttt} + 10*u*u1_{t}*u_{tttt} + 10*u1*u_{tttt}*u_{t} + 20*u1*u_{ttt}*u_{tt} - u1_{ttttt} + 20*u1_{ttt}*u_{t}**2 + 60*u1_{tt}*u_{tt}*u_{t} + 40*u1_{t}*u_{ttt}*u_{t} + 30*u1_{t}*u_{tt}**2)/1440 + (6734*u1_{tttttt} + 6761*u_{ttttt})/1440

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
\frac{7 \mu \left(u^{2} u1_{t} + 2 u u_{1} u_{t} - u1_{t}\right)}{2} + \frac{7 \left(u1_{tt} + u_{t}\right)}{2}
\right)
+h^2\left(
\frac{37 \mu \left(u^{2} u1_{tt} + 2 u u_{1} u_{tt} + 4 u u1_{t} u_{t} + 2 u_{1} u_{t}^{2} - u1_{tt}\right)}{6} + \frac{37 \left(u1_{ttt} + u_{tt}\right)}{6}
\right)
+h^3\left(
\frac{175 \mu \left(u^{2} u1_{ttt} + 2 u u_{1} u_{ttt} + 6 u u1_{tt} u_{t} + 6 u u1_{t} u_{tt} + 6 u_{1} u_{tt} u_{t} - u1_{ttt} + 6 u1_{t} u_{t}^{2}\right)}{24} + \frac{175 \left(u1_{tttt} + u_{ttt}\right)}{24}
\right)
+h^4\left(
\frac{781 \mu \left(u^{2} u1_{tttt} + 2 u u_{1} u_{tttt} + 8 u u1_{ttt} u_{t} + 12 u u1_{tt} u_{tt} + 8 u u1_{t} u_{ttt} + 8 u_{1} u_{ttt} u_{t} + 6 u_{1} u_{tt}^{2} - u1_{tttt} + 12 u1_{tt} u_{t}^{2} + 24 u1_{t} u_{tt} u_{t}\right)}{120} + \frac{781 \left(u1_{ttttt} + u_{tttt}\right)}{120}
\right)
+h^5\left(
\frac{6761 \mu \left(u^{2} u1_{ttttt} + 2 u u_{1} u_{ttttt} + 10 u u1_{tttt} u_{

In [61]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


3*\mu**5*u1*(u - 1)**5*(u + 1)**5/160 + 3*\mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/160 - 3*\mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/40 - 3*\mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/160 + 3*\mu*u1*(69*u**2 - 22*u1**2 - 3)/160 + 3*u/160

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{3 \mu^{5} u_{1} \left(u - 1\right)^{5} \left(u + 1\right)^{5}}{160} + \frac{3 \mu^{4} u \left(u - 1\right)^{3} \left(u + 1\right)^{3} \left(u^{2} - 52 u_{1}^{2} - 1\right)}{160} - \frac{3 \mu^{3} u_{1} \left(u - 1\right) \left(u + 1\right) \left(20 u^{4} - 50 u^{2} u_{1}^{2} - 21 u^{2} + 16 u_{1}^{2} + 1\right)}{40} - \frac{3 \mu^{2} u \left(29 u^{4} - 250 u^{2} u_{1}^{2} - 32 u^{2} + 60 u_{1}^{4} + 150 u_{1}^{2} + 3\right)}{160} + \frac{3 \mu u_{1} \left(69 u^{2} - 22 u_{1}^{2} - 3\right)}{160} + \frac{3 u}{160}
\right)


In [62]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-3*\mu**6*u1*(u - 1)**6*(u + 1)**6/160 - 3*\mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 114*u1**2 - 1)/160 + 3*\mu**4*u1*(u - 1)**2*(u + 1)**2*(193*u**4 - 964*u**2*u1**2 - 198*u**2 + 244*u1**2 + 5)/160 + 3*\mu**3*u*(20*u**6 - 395*u**4*u1**2 - 41*u**4 + 260*u**2*u1**4 + 562*u**2*u1**2 + 22*u**2 - 192*u1**4 - 167*u1**2 - 1)/40 - 9*\mu**2*u1*(119*u**4 - 176*u**2*u1**2 - 78*u**2 + 10*u1**4 + 36*u1**2 + 1)/80 - 9*\mu*u*(23*u**2 - 68*u1**2 - 1)/160 + 3*u1/160

\mu u_{1} \left(u - 1\right) \left(u + 1\right) + u + u1_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{3 \mu^{6} u_{1} \left(u - 1\right)^{6} \left(u + 1\right)^{6}}{160} - \frac{3 \mu^{5} u \left(u - 1\right)^{4} \left(u + 1\right)^{4} \left(u^{2} - 114 u_{1}^{2} - 1\right)}{160} + \frac{3 \mu^{4} u_{1} \left(u - 1\right)^{2} \left(u + 1\right)^{2} \left(193 u^{4} - 964 u^{2} u_{1}^{2} - 198 u^{2} + 244 u_{1}^{2} + 5\right)}{160} + \frac{3 \mu^{3} u \left(20 u^{6} - 395 u^{4} u_{1}^{2} - 41 u^{4} + 260 u^{2} u_{1}^{4} + 562 u^{2} u_{1}^{2} + 22 u^{2} - 192 u_{1}^{4} - 167 u_{1}^{2} - 1\right)}{40} - \frac{9 \mu^{2} u_{1} \left(119 u^{4} - 176 u^{2} u_{1}^{2} - 78 u^{2} + 10 u_{1}^{4} + 36 u_{1}^{2} + 1\right)}{80} - \frac{9 \mu u \left(23 u^{2} - 68 u_{1}^{2} - 1\right)}{160} + \frac{3 u_{1}}{160}
\right)


In [63]:
replace(str(f2[5].collect(mu)))

'mu**6*(-3*y[0]**12*y[1]/160 + 9*y[0]**10*y[1]/80 - 9*y[0]**8*y[1]/32 + 3*y[0]**6*y[1]/8 - 9*y[0]**4*y[1]/32 + 9*y[0]**2*y[1]/80 - 3*y[1]/160) + mu**5*(-3*y[0]**11/160 + 171*y[0]**9*y[1]**2/80 + 3*y[0]**9/32 - 171*y[0]**7*y[1]**2/20 - 3*y[0]**7/16 + 513*y[0]**5*y[1]**2/40 + 3*y[0]**5/16 - 171*y[0]**3*y[1]**2/20 - 3*y[0]**3/32 + 171*y[0]*y[1]**2/80 + 3*y[0]/160) + mu**4*(579*y[0]**8*y[1]/160 - 723*y[0]**6*y[1]**3/40 - 219*y[0]**6*y[1]/20 + 1629*y[0]**4*y[1]**3/40 + 891*y[0]**4*y[1]/80 - 1089*y[0]**2*y[1]**3/40 - 39*y[0]**2*y[1]/10 + 183*y[1]**3/40 + 3*y[1]/32) + mu**3*(3*y[0]**7/2 - 237*y[0]**5*y[1]**2/8 - 123*y[0]**5/40 + 39*y[0]**3*y[1]**4/2 + 843*y[0]**3*y[1]**2/20 + 33*y[0]**3/20 - 72*y[0]*y[1]**4/5 - 501*y[0]*y[1]**2/40 - 3*y[0]/40) + mu**2*(-1071*y[0]**4*y[1]/80 + 99*y[0]**2*y[1]**3/5 + 351*y[0]**2*y[1]/40 - 9*y[1]**5/8 - 81*y[1]**3/20 - 9*y[1]/80) + mu*(-207*y[0]**3/160 + 153*y[0]*y[1]**2/40 + 9*y[0]/160) + 3*y[1]/160'

In [64]:
end = time.time()
print(end - start)

1387.1660599708557
